Importing all required libraries and modules, initializing cuda for model training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
from torch.utils.data import DataLoader, TensorDataset
import joblib

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Developing the Model class

In [ ]:
#Hyperparameters
epochs = 10
embed_size = 128
hidden_size = 256
num_layers = 2
seq_length = 20
lr = 0.001
batch_size = 32

In [ ]:
class StoryModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(StoryModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, hidden):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden
    
    def init_hidden(self, batchs_size):
        hidden = (torch.zeros(num_layers, batchs_size, hidden_size).to(device),
                  torch.zeros(num_layers, batchs_size,hidden_size).to(device))
        return hidden

Data Preparation: 
Here i use the gutenberg text file from nltk library "Alice in Wonderland" by lewis caroll

note print statements and values given

In [ ]:
stories = [gutenberg.raw("carroll-alice.txt"),
        gutenberg.raw("shakespeare-hamlet.txt"),
        ]

# confirm file is loaded
assert stories is not None, "files not found"

#combining stories to form one text corpus seperated by token
combined_stories = "<sep>".join(stories)

# all words will now be converted to lower forms and split i.e tokenized
#tokens = text.lower().split()
tokens = word_tokenize(combined_stories.lower())
#print(len(tokens))


#Creating a word vovabulary based in this tokens
special_tokens = ["<sep>", "<unk>", "<eos>"]
vocab = list(set(tokens + special_tokens))
word_index = {word: idx for idx, word in enumerate(vocab)}
index__word = {idx: word for word, idx in word_index.items()}
vocab_size = len(vocab)

print(len(word_index))

#converting tokens to indices
#input_seq = [word_index[word] for word in tokens]
#here the list comprehension checks and gets the indexed value for words in the word_index if nor in word_index
# assigns the <unk> index to the word
input_seq = [word_index.get(word, word_index["<unk>"]) for word in tokens] 
print(input_seq[:10])
print(len(input_seq))

Model creation and parameters

In [ ]:
#seting device hardware config to train module on either cpu or gpu
model = StoryModel(vocab_size, embed_size, hidden_size,num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

Here, a function is created to split the tokens into input and output data

In [ ]:
def create_sequences(input_seq, seq_length):
    input_data = []
    output_data = []
    
    for i in range(len(input_seq) - seq_length):
        input_data.append(input_seq[i:i+seq_length])
        output_data.append(input_seq[i+seq_length])
    return torch.tensor(input_data), torch.tensor(output_data)

#Create sequences
input_data, output_data = create_sequences(input_seq, seq_length)

#just to check the dim of squences
print(input_data.ndim)
print(output_data.ndim)

Training the model

In [ ]:
dataset = TensorDataset(input_data, output_data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
print("Please Wait")
print("Starting Model Training.......")
for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (batch_input, batch_output) in enumerate(dataloader):
        batch_input, batch_output = batch_input.to(device), batch_output.to(device)
        hidden = model.init_hidden(batch_input.size(0))  # Initialize hidden state for current batch size

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        output, hidden = model(batch_input, hidden)

        # Use only the final timestep output
        output = output[:, -1, :]

        # Compute and record loss
        loss = criterion(output, batch_output)

        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Print average loss for each epoch
    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.2f}')

print("Model Trained")


Saving the model 

In [13]:
model_filename = 'StoryModel.pkl'
joblib.dump(model, model_filename)
print("Model Created!!!")

Model Created!!!


Define function to generate story

In [26]:

def generate_story(model, prompt, max_length=100, temperature=1.0):
    # Convert the prompt to indices
    words = prompt.lower().split()
    input_seq = torch.tensor([word_index.get(word, word_index['<unk>']) for word in words], device=device).unsqueeze(0)
    
    # Initialize hidden state
    hidden = model.init_hidden(1)
    
    # Generate words until max length
    for _ in range(max_length):
        output, hidden = model(input_seq, hidden)
        
        # Select the last word's output in the sequence
        last_word_logits = output[:, -1, :] / temperature
        probabilities = torch.softmax(last_word_logits, dim=-1).squeeze()
        next_word_idx = torch.multinomial(probabilities, 1).item()
        next_word = index__word.get(next_word_idx, "<unk>")
        words.append(next_word)
        input_seq = torch.tensor([[next_word_idx]], device=device)
        if next_word == "<sep>":  # Stop generation at story boundary
            break
    
    return ' '.join(words)




Define code for cli interaction

In [27]:
def start_story():
    print("########## ''''''''''' ############")
    print("Hi am Tobbie a story generator!!!!")
    print("########## ''''''''''' ############")
    prompt = input("What story would you like to here: \n")
    while True:
        generated = generate_story(model, prompt)
        print(f"\nLet's begin:\n\n{generated}")
        choice = input("\nWhat's next? (Type a new prompt or 'quit' to exit): ").strip()
        if choice.lower() == 'quit':
            print("BYEEE!")
            break
        prompt = choice

In [28]:
start_story()

########## ''''''''''' ############
Hi am Tobbie a story generator!!!!
########## ''''''''''' ############

Let's begin:

alice first , 'it in those of one eye near the time , in very leaves that lay made her mark ] out of my weaknesse , and with eyes like a upon a thousand fire . an old now ? laer . it mocke , when seeing losse within ? guil . aye , but answers it is him then . how long horatio 't ? osr . iudgement man in clouds , go to himselfe . enter hamlet and barnardo . qu . my necessaries you stand ; so sir , is good by my say lady , let
BYEEE!
